# Software tutorial

In [1]:
import numpy as np
from astropy import units as u

## Defining sources

In [2]:
from python.source.parameter import Parameter
from python.source.source import Sources, PointSource

We can start by defining the high-level source parameters, these are essentially the hyperparameters of the hierarchical model.

In [3]:
Parameter.clear_registry()
index = Parameter(2.4, "index", fixed=False, par_range=(1.0, 4))
L = Parameter(1E47 * (u.erg / u.s), "luminosity", fixed=True, par_range=(0, 1E60))
diffuse_norm = Parameter(1.44E-14 /u.GeV/u.m**2/u.s, "diffuse_norm", fixed=True, 
                         par_range=(0, np.inf))
Enorm = Parameter(1E5 * u.GeV, "Enorm", fixed=True)
Emin = Parameter(1E5 * u.GeV, "Emin", fixed=True)
Emax = Parameter(1E8 * u.GeV, "Emax", fixed=True)
Emin_det = Parameter(1E5 * u.GeV, "Emin_det", fixed=True)

Point sources can be defined either manually or from an HDF5 file. We can use the `Sources` container to keep track of sources and pass them around the software.

In [4]:
point_source = PointSource.make_powerlaw_source("test", np.deg2rad(20)*u.rad,
                                                np.pi*u.rad, 
                                                L, index, 0.5, Emin, Emax)
my_sources = Sources()
my_sources.add(point_source) # single point source

In [5]:
# alternatively load point sources from file
my_sources = Sources()
source_file = "../dev/statistical_model/data/test_SFR_pop.h5"
point_sources = PointSource.make_powerlaw_sources_from_file(source_file, L, 
                                                            index, Emin, Emax)
my_sources.add(point_sources)
my_sources.select_below_redshift(0.46)

There are helper methods to automatically set the diffuse and atmospheric components based on the definition of the point sources..

In [6]:
my_sources.add_diffuse_component(diffuse_norm, Enorm.value) # auto diffuse component 
my_sources.add_atmospheric_component() # auto atmo component

We should always add the diffuse component second-to-last and the atmospheric component last for consistency with later simulation/fit inputs. We can make sure this is done by calling the `organise()` method.

In [7]:
my_sources.organise()

We can also check some useful properties of our sources...

In [8]:
my_sources.total_flux_int()

<Quantity 2.20421896e-08 1 / (m2 s)>

In [9]:
my_sources.associated_fraction()

<Quantity 0.13348525>

## Precomputation

This module performs some useful calculations that are passed into the simulation/fit later on. All we need is a bunch of `Sources` and a `DetectorModel` to compute how many neutrinos we expect to detect from the different source components. This code can take a while to run.

In [10]:
from python.precomputation import ExposureIntegral
from python.detector_model import NorthernTracksDetectorModel

In [11]:
exposure_integral = ExposureIntegral(my_sources, NorthernTracksDetectorModel)